In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [9]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


### Chat Model - Gemini GenAI

In [2]:
%pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

### Embeddings model - HuggingFace

In [4]:
%pip install -qU langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

### Memory Vector Model - LangChain Core

In [3]:
%pip install -qU langchain-core

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [1]:
%pip install --upgrade --quiet  langchain langchain-community azure-ai-documentintelligence

Note: you may need to restart the kernel to use updated packages.


In [5]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

# Load and chunk contents of the blog
file_path = "test.pptx"
endpoint = "https://ctse-chatbot-1.cognitiveservices.azure.com/"
key = getpass.getpass("Enter API key for Document Intelligence API: ")
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=key, file_path=file_path, api_model="prebuilt-layout"
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
response = graph.invoke({"question": "List 3 generally reusable solution to a recurring problem?"})
print(response["answer"])

Three generally reusable solutions to a recurring problem are design patterns, templates, and best practices. Design patterns offer a template to solve the problem. They also improve developer communication.
